# Import the necessary stuff

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
#mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_face_mesh = mp.solutions.face_mesh
mp_drawing_styles = mp.solutions.drawing_styles

### code for extracting keypoints

In [3]:
def extract_face_keypoints(results):
    face = np.array([[res.x, res.y, res.z] for res in np.array(results.multi_face_landmarks)[0].landmark]).flatten() if results.multi_face_landmarks else np.zeros(468*3)
    return np.concatenate([face])

In [8]:
#def mediapipe_detection(image, model):
#    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
#    image.flags.writeable = False                  # Image is no longer writeable
#    results = model.process(image)                 # Make prediction
#    image.flags.writeable = True                   # Image is now writeable 
#    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
#    return image, results


# Tester 

### set where we must looking for pictures

In [5]:
DATA_PATH = os.path.join('E:\Document\FIDs_NEW') 

# Actions that we try to detect
actions = np.array(['F0001', 'F0002', 'F0003'])

In [6]:
def get_filepaths(directory):
    """
    This function will generate the file names in a directory tree by walking the tree either top-down or bottom-up. 
    For each directory in the tree rooted at directory top (including top itself), it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".jpg"):
                # Join the two strings in order to form the full filepath.
                filepath = os.path.join(root, filename)
                file_paths.append(filepath)  # Add it to the list.

    return file_paths  # Self-explanatory.

# Run the above function and store its results in a variable.   
full_file_paths = get_filepaths(DATA_PATH)
pic_number = len(full_file_paths)

In [7]:
full_file_paths[1] # test for path pictures

'E:\\Document\\FIDs_NEW\\F1000\\MID9\\P10578_face4.jpg'

In [ ]:
image_nameList = [string[-16:] for string in full_file_paths]
image_nameList

In [12]:
image_nameList = [string[:-4] for string in full_file_paths]
image_nameList[1]

'E:\\Document\\FIDs_NEW\\F0001\\MID1\\P00002_face3'

## this for collecting the keypoints (looking a file in folder directory, then loop the code) 

In [22]:
for i in range(pic_number):
                  
      # For static images:
      IMAGE_FILES = [full_file_paths[i]]
      drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
      with mp_face_mesh.FaceMesh(
          static_image_mode=True,
          max_num_faces=1,
          refine_landmarks=True,
          min_detection_confidence=0.5) as face_mesh:
        for idx, file in enumerate(IMAGE_FILES):
          image = cv2.imread(file)
          # Convert the BGR image to RGB before processing. 
          results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

          # Print and draw face mesh landmarks on the image.
          if not results.multi_face_landmarks:
            continue
          annotated_image = image.copy()
          for face_landmarks in results.multi_face_landmarks:
            #print('face_landmarks:', face_landmarks)
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_tesselation_style())
            #mp_drawing.draw_landmarks(
            #    image=annotated_image,
            #    landmark_list=face_landmarks,
            #    connections=mp_face_mesh.FACEMESH_CONTOURS,
            #    landmark_drawing_spec=None,
            #    connection_drawing_spec=mp_drawing_styles
            #    .get_default_face_mesh_contours_style())
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_IRISES,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_iris_connections_style())
          # cv2.imwrite('videos' + str(idx) + '.png', annotated_image)  # for save image with keypoints
          keypoints = extract_face_keypoints(results)  # for extract keypoints into array
          npy_path = os.path.join(image_nameList[i])
          np.save(npy_path, keypoints)



## this code used if we will use a video (not yet added features to dettect files in directory so must do it manually)

In [3]:
# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture('videos/ss_003_2.mp4')

with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while True:
        #cap.isOpened():
        success, image = cap.read()

        # Flip the image horizontally for a selfie-view display.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False

        # Detect the face landmarks
        results = face_mesh.process(image)
        
        # To improve performance
        image.flags.writeable = True
        
        # Convert back to the BGR color space
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                   .get_default_face_mesh_tesselation_style())
#                mp_drawing.draw_landmarks(
#                    image=image,
#                    landmark_list=face_landmarks,
#                    connections=mp_face_mesh.FACEMESH_CONTOURS,
#                    landmark_drawing_spec=None,
#                    connection_drawing_spec=mp_drawing_styles
#                    .get_default_face_mesh_contours_style())
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style())
                
        # Display the image
        cv2.imshow('MediaPipe FaceMesh', image)
        
        

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [17]:
# tester code for collect array

ress = np. array(results.multi_face_landmarks)
resss = ress[0].landmark
faces = np.array([[res.x, res.y, res.z, res.visibility] for res in resss])
faces[1]
#face = np.array([[res.x, res.y, res.z] for res in results.multi_face_landmarks])


array([ 0.49551976,  0.43732437, -0.06846248,  0.        ])

In [ ]:
'''
export const MESH_ANNOTATIONS: {[key: string]: number[]} = {
  silhouette: [
    10,  338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288,
    397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136,
    172, 58,  132, 93,  234, 127, 162, 21,  54,  103, 67,  109
  ],

  lipsUpperOuter: [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291],
  lipsLowerOuter: [146, 91, 181, 84, 17, 314, 405, 321, 375, 291],
  lipsUpperInner: [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308],
  lipsLowerInner: [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308],

  rightEyeUpper0: [246, 161, 160, 159, 158, 157, 173],
  rightEyeLower0: [33, 7, 163, 144, 145, 153, 154, 155, 133],
  rightEyeUpper1: [247, 30, 29, 27, 28, 56, 190],
  rightEyeLower1: [130, 25, 110, 24, 23, 22, 26, 112, 243],
  rightEyeUpper2: [113, 225, 224, 223, 222, 221, 189],
  rightEyeLower2: [226, 31, 228, 229, 230, 231, 232, 233, 244],
  rightEyeLower3: [143, 111, 117, 118, 119, 120, 121, 128, 245],

  rightEyebrowUpper: [156, 70, 63, 105, 66, 107, 55, 193],
  rightEyebrowLower: [35, 124, 46, 53, 52, 65],

  rightEyeIris: [473, 474, 475, 476, 477],

  leftEyeUpper0: [466, 388, 387, 386, 385, 384, 398],
  leftEyeLower0: [263, 249, 390, 373, 374, 380, 381, 382, 362],
  leftEyeUpper1: [467, 260, 259, 257, 258, 286, 414],
  leftEyeLower1: [359, 255, 339, 254, 253, 252, 256, 341, 463],
  leftEyeUpper2: [342, 445, 444, 443, 442, 441, 413],
  leftEyeLower2: [446, 261, 448, 449, 450, 451, 452, 453, 464],
  leftEyeLower3: [372, 340, 346, 347, 348, 349, 350, 357, 465],

  leftEyebrowUpper: [383, 300, 293, 334, 296, 336, 285, 417],
  leftEyebrowLower: [265, 353, 276, 283, 282, 295],

  leftEyeIris: [468, 469, 470, 471, 472],

  midwayBetweenEyes: [168],

  noseTip: [1],
  noseBottom: [2],
  noseRightCorner: [98],
  noseLeftCorner: [327],

  rightCheek: [205],
  leftCheek: [425]
link for picture https://github.com/google/mediapipe/blob/a908d668c730da128dfa8d9f6bd25d519d006692/mediapipe/modules/face_geometry/data/canonical_face_model_uv_visualization.png

478 * 3 = 1434 keypoints


kinsip
mother
daughter
son
father

1	=	anak dari
2	=	saudara dari
3	=	cucu dari
4	=	orangtua dari
5	=	istri/suami
6	=	kakek/nenek dari

MID	1	2	3	4	5	6						Gender	Name
1	  0	5	4	4	4	1						Male	  Juan Esteban
2	  5	0	4	4	4	0						Female	karen
3	  1	1	0	2	2	3						Female	Paloma
4	  1	1	2	0	2	3						Male	  Dante
5	  1	1	2	2	0	3						Female	Luna
6	  4	0	6	6	6	0						Female	Alicia



'''


In [16]:
 #input:===================================================================================
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture('videos/ss_003_2.mp4')
frame_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    for frame_num in range(frame_length):
        #cap.isOpened():
        ret, image = cap.read()

        # Flip the image horizontally for a selfie-view display.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False

        # Detect the face landmarks
        results = face_mesh.process(image)
        
        # To improve performance
        image.flags.writeable = True

        # Convert back to the BGR color space
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                   .get_default_face_mesh_tesselation_style())

                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style())

        # Display the image
        if frame_num == 0:
            cv2.putText(image, 'STARTING COLLECTION', (120,200),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
            cv2.putText(image, 'Collecting frames num {} Video {}'.format(frame_num, 'test'), (70,50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            # Show to screen
            cv2.imshow('MediaPipe FaceMesh', image)
            cv2.waitKey(500)
        else:
            cv2.putText(image, 'Collecting frames num {} Video {} '.format(frame_num, 'test'), (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            # Show to screen
            cv2.imshow('MediaPipe FaceMesh', image)
                
        keypoints = extract_face_keypoints(results)
        npy_path = os.path.join(DATA_PATH, "male", str(frame_num))
        np.save(npy_path, keypoints)


        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [15]:
loaderr = np.load('E:\Document\FIDs_NEW\F0001\MID1\P00002_face3.npy') # to check our keypoints that we collects
len(loaderr)   #lengt each array about 478 * 3 = 1434 

1434

In [10]:
loaderr


array([ 0.48946536,  0.674936  , -0.06828458, ...,  0.70081866,
        0.31247622, -0.04326841])